In [5]:
import pygame
import glob
import os
import vlc
import time

In [6]:
def get_sound_name_from_address(address):
    """
    :param address: address to audio file
    """
    return address.split('/')[1].rstrip('.wav')

def get_sound_files_from_folder(folder):
    """
    :param folder: address of folder containing audio files
    """
    addresses = glob.glob(folder+"*.wav")
    file_dict = {}
    for address in addresses:
        file_dict[get_sound_name_from_address(address)] = address
    return file_dict

def play_sound(sound):
    print("Loading", sound)
    instance = vlc.Instance()              # Create new instance
    media = instance.media_new(sound) # Load media file to instance
    player = vlc.MediaPlayer()            # Create players
    player.set_media(media)
    player.play()

def stop_sound(sound):
    instance = vlc.Instance()              # Create new instance
    media = instance.media_new(sound) # Load media file to instance
    player = vlc.MediaPlayer()            # Create players
    player.vlm_del_media(media)

def mins_to_ticks(time):
    m, s = time.split(':')
    return int(1000 * (int(m) * 60 + float(s)))

def merge_dicts(dict1, dict2):
    return(dict2.update(dict1))

In [7]:
pygame.init()             # Initialise pygame  
pygame.display.set_mode((600, 400)) # Initialise display window   

# Load sounds
sound_files = get_sound_files_from_folder(folder = '4 Channel Bounces/')
merge_dicts(get_sound_files_from_folder(folder = '4 Channel Fight/'), sound_files)
start_time = pygame.time.get_ticks()

# Set length of button events
BUTTON_WAIT = 1.5

# Define user events
Q1_event = pygame.USEREVENT
Q3_event = pygame.USEREVENT + 1
leg_event = pygame.USEREVENT + 2
Q4_event = pygame.USEREVENT + 3
Q5_event = pygame.USEREVENT + 4
death_event = pygame.USEREVENT + 5
fight_event = pygame.USEREVENT + 6
no_hit_1_event = pygame.USEREVENT + 7
no_hit_2_event = pygame.USEREVENT + 8
no_hit_3_event = pygame.USEREVENT + 9

# Set timers for user events
'''
    There are x types of events:
    - question events (timer and button activation)
    - silent events (timer with no button)
    - punches (button with count and sleep)
'''
pygame.time.set_timer(Q1_event, mins_to_ticks('0:14.5'))
pygame.time.set_timer(Q3_event, mins_to_ticks('0:44.5'))
pygame.time.set_timer(leg_event, mins_to_ticks('1:33'))
pygame.time.set_timer(Q4_event, mins_to_ticks('2:18'))
pygame.time.set_timer(Q5_event, mins_to_ticks('2:53.5'))
pygame.time.set_timer(death_event, mins_to_ticks('3:28.5'))
pygame.time.set_timer(fight_event, mins_to_ticks('3:42'))
pygame.time.set_timer(no_hit_1_event, mins_to_ticks('3:54'))
pygame.time.set_timer(no_hit_2_event, mins_to_ticks('4:04'))
pygame.time.set_timer(no_hit_3_event, mins_to_ticks('4:12'))

play_sound(sound_files['AUDIO_1'])

'''=========================================================================='''

# Functions which need start_ticks
def time_is_between(start, end):
    start_ticks = mins_to_ticks(start)
    end_ticks = mins_to_ticks(end)
    return (pygame.time.get_ticks() - start_time) >= start_ticks and (pygame.time.get_ticks() - start_time) <= end_ticks

def button_event(time, wait=BUTTON_WAIT):
    start = time
    m, s = time.split(':')
    if wait != BUTTON_WAIT:
        wait_m, wait_s = wait.split(':')
        end = f'{int(m)+int(wait_m)}:{float(s)+float(wait_s)}'
    else:
        end = f'{int(m)}:{float(s)+wait}'
    if time_is_between(time, end):
        return True

'''=========================================================================='''

Q1 = Q2 = Q3 = Q4 = Q5 = False
punches = 0

while True: # Infinite loop
    for event in pygame.event.get():  # For any event (i.e. key press)
        if event.type == pygame.QUIT: # if event is a quit event, (i.e. alt-F4)
            pygame.quit()             # quit game
        if event.type == pygame.KEYDOWN:           # If a key is pressed
            if event.key == pygame.K_SPACE:            # and the key is d
                if button_event('0:13'):
                    Q1 = True
                if button_event('0:16'):
                    Q2 = True
                if button_event('0:43'):
                    Q3 = True
                if button_event('2:16.5'):
                    Q4 = True
                if button_event('2:52.5'):
                    Q5 = True
                if button_event('3:44', '1:00'):
                    if punches == 0:
                        play_sound(sound_files['AUDIO_17_PUNCH_1'])
                    elif punches == 1:
                        play_sound(sound_files['AUDIO_23_PUNCH_2'])
                    elif punches == 2:
                        play_sound(sound_files['AUDIO_24_PUNCH_3'])
                    elif punches == 3:
                        play_sound(sound_files['AUDIO_25_PUNCH_4'])
                    elif punches == 4:
                        play_sound(sound_files['AUDIO_26_PUNCH_5'])
                    elif punches == 5:
                        play_sound(sound_files['AUDIO_27_PUNCH_6'])
                    elif punches == 6:
                        play_sound(sound_files['AUDIO_28_PUNCH_7'])
                    punches += 1
                    time.sleep(5)
                print('Key pressed')
        if event.type == Q1_event:
            print('Q1 response:', Q1)
            if Q1:
                play_sound(sound_files['AUDIO_2'])
            if not Q1:
                play_sound(sound_files['AUDIO_3'])
            pygame.time.set_timer(Q1_event, 0) 
        if event.type == Q3_event:
            print('Q3 response:', Q3)
            if Q3:
                play_sound(sound_files['AUDIO_4'])
            if not Q3:
                play_sound(sound_files['AUDIO_5'])
            pygame.time.set_timer(Q3_event, 0)
        if event.type == leg_event:
            if Q2:
                if Q1: 
                    play_sound(sound_files['AUDIO_6'])
                if not Q1:
                    play_sound(sound_files['AUDIO_7'])
            if not Q2:
                play_sound(sound_files['AUDIO_8'])
            pygame.time.set_timer(leg_event, 0)
        if event.type == Q4_event:
            print('Q4 response:', Q4)
            if Q4:
                play_sound(sound_files['AUDIO_9'])
            if not Q4:
                play_sound(sound_files['AUDIO_10'])
            pygame.time.set_timer(Q4_event, 0)
        if event.type == Q5_event:
            print('Q5 response:', Q5)
            if not Q4:
                if Q5:
                    play_sound(sound_files['AUDIO_11'])
                if not Q5:
                    play_sound(sound_files['AUDIO_12'])
            pygame.time.set_timer(Q5_event, 0)
        if event.type == death_event:
            if Q2:
                if Q1:
                    play_sound(sound_files['AUDIO_13'])
                if not Q1:
                    play_sound(sound_files['AUDIO_14'])
            if not Q2:
                play_sound(sound_files['AUDIO_15'])
            pygame.time.set_timer(death_event, 0)
        if event.type == fight_event:
            play_sound(sound_files['AUDIO_16_fight_base'])
            pygame.time.set_timer(fight_event, 0)
        if event.type == no_hit_1_event:
            if punches == 0:
                play_sound(sound_files['AUDIO_18_HIT_ME_1'])
            pygame.time.set_timer(no_hit_1_event, 0)
        if event.type == no_hit_2_event:
            if punches == 0:
                play_sound(sound_files['AUDIO_19_HIT_ME_2'])
            pygame.time.set_timer(no_hit_2_event, 0)
        if event.type == no_hit_3_event:
            if punches == 0:
                play_sound(sound_files['AUDIO_20_HIT_ME_3'])
            pygame.time.set_timer(no_hit_3_event, 0)
        pygame.display.update()                    # and update display


Loading 4 Channel Bounces/AUDIO_1.wav


error: video system not initialized

In [40]:
pygame.init()             # Initialise pygame  
screen = pygame.display.set_mode((600, 400)) # Initialise display window   

red = (255, 0, 0)
black = (0, 0, 0)

BUTTON_WAIT = '0:01.5'
TIME_BUFFER = '0:01'

button_on_event = pygame.USEREVENT
button_off_event = pygame.USEREVENT + 1
sound_cue_event = pygame.USEREVENT + 2

# Load sounds
sound_files = get_sound_files_from_folder(folder = '4 Channel Bounces/')
merge_dicts(get_sound_files_from_folder(folder = '4 Channel Fight/'), sound_files)
start_time = pygame.time.get_ticks()
print(start_time)

def add_times(*times):
    mins = secs = 0
    for time in times:
        m, s = time.split(':')
        mins += int(m)
        secs += float(s)
    return f'{mins}:{secs}'

def create_event(event, start, duration=BUTTON_WAIT, button=False):
    pygame.time.set_timer(event, mins_to_ticks(start), loops=1)
    if button:
        pygame.time.set_timer(button_off_event, mins_to_ticks(add_times(start, duration)), loops=1)

# Functions which need start_ticks
def time_is_between(start, end):
    start_ticks = mins_to_ticks(start)
    end_ticks = mins_to_ticks(end)
    return (pygame.time.get_ticks() - start_time) >= start_ticks and (pygame.time.get_ticks() - start_time) <= end_ticks

def get_button_press_starting(time, wait=BUTTON_WAIT):
    start = time
    m, s = time.split(':')
    wait_m, wait_s = wait.split(':')
    end = f'{int(m)+int(wait_m)}:{float(s)+float(wait_s)}'
    if time_is_between(time, end):
        return True

def turn_on_button():
    print('Button on')
    screen.fill(red)
    pygame.display.flip()
    pygame.display.update()  

def turn_off_button():
    print('Button off')
    screen.fill(black)
    pygame.display.flip()
    pygame.display.update() 

create_event(button_on_event, '0:4.5', button=True)   # Q1
create_event(button_on_event, '0:6', button=True)     # Q2

while True: # Infinite loop
    for event in pygame.event.get():  # For any event (i.e. key press)
        if event.type == pygame.QUIT: # if event is a quit event, (i.e. alt-F4)
            pygame.quit()             # quit game            
        if event.type == pygame.KEYDOWN:           # If a key is pressed
            if event.key == pygame.K_SPACE:            # and the key is d
                if get_button_press_starting('0:13'):
                    Q1 = True
                if get_button_press_starting('0:16'):
                    Q2 = True
        if event.type == button_on_event:
            turn_on_button() 
            
        if event.type == button_off_event:
            turn_off_button()
        if event.type == sound_cue_event:
            if time_event('0:13', TIME_WINDOW):
                if Q1: play_sound(sound_files['AUDIO_2'])
                else: play_sound(sound_files['AUDIO_3'])
        pygame.display.update()                    # and update display


122
Button on
Button off


error: video system not initialized

1. Play first track, set `button_on_timer` for `'0:13'`
2. At `'0:13'` turn on button, set  `button_off_timer` for `'0:01.5'`
3. At `'0:14.5'` turn off button, play audio, set timer for (`'0:16'-time_now`)
4. At `'0:16'` turn on button, set timer for `'0:1.5'`
5. At `'0:17.5'` turn off button, set timer for (`'0:43'-time_now`)

In [38]:
import numpy as np
import pandas as pd

def get_args(event):
    name = event['name']
    
# Name, time, wait, response, audio, turn_button_on
array = [
    ['Q1', '0:13', None, False, True],
    ['Q1_response', '0:14.5', None, True, False],
    ['Q2', '0:16', None, False, True]
]
array = np.array(array)
names = ['Q1',  'Q1_response', 'Q2', 'Q2_response']
times = ['0:13','0:14', '0:16', '0:']
responses = np.empty_like(names)
audios = [False, True, False, False]
buttons = [True, False, True, False]

d = {'name':array[:,0], 
     'time':array[:,1],
     'response':array[:,2],
     'audio':array[:,3],
     'button':array[:,4],
    }
events = pd.DataFrame(data=d)
get_args(events.loc[0,:])
events



,name,time,response,audio,button
0,Q1,0:13,None,False,True
1,Q1_response,0:14.5,None,True,False
2,Q2,0:16,None,False,True
